In [6]:

%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint
from joblib import dump, load

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd


import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.linear_model import LogisticRegression


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'

r = redis.Redis(host="127.0.0.1", port=6379, db=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# from register_check import RegisterCheck
# from pprint import pprint
# m = RegisterCheck()

# pprint(m.score_user('yryryr@gmail.com', '081367758790', '10.4.77.18', 'akhiyarwaladi'))

In [8]:
def encode(s):
    
    i = 0
    t = ''
    while i < len(s):
        if s[i].isupper():
            t += 'U'
        elif s[i].islower():
            t += 'L'
        elif s[i].isdigit():
            t += 'D'
        else:
            t += 'O'
        i += 1
        
    return t

def short_encode(s):
    i = 0
    s = encode(s)
    curr = ''
    t = ''
    
    while i < len(s):
        if curr != s[i]:
            t += s[i]
            curr = s[i]
        
        i += 1
    return t

def score_sus_email(s):

    email_input = s
    email_input = email_input.split('@')[0]
    
    
    
    pat_count_digit = encode(email_input).count('D')
    pat_encode = short_encode(email_input)
    
    if pat_encode == 'LD' and pat_count_digit >= 4:
        return 1
    elif pat_encode == 'D':
        return 1
    return 0
    
score_sus_email('123312313@gmail.com')

1

In [9]:
from gibberish_detector import detector

path_detector_indo = '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/indo_news.model'
Detector = detector.create_from_model(
    path_detector_indo
)
from nostril import nonsense


def score_gibberish(input_email):
    split_email = input_email.split('@')[0]
    #print(split_email)

    flag_gibberish1 = False
    
    try:
        if Detector.is_gibberish(split_email):
            flag_gibberish1 = True
    except Exception as e:
        pass
    
    flag_gibberish2 = False
    
    try:
        if nonsense(split_email):
            flag_gibberish2 = True
    except Exception as e:
        pass


    if flag_gibberish1 and flag_gibberish2:
        return 1
    else:
        return 0

email = 'qywyeqeyqw123@gmail.com'
score_gibberish(email)

1

In [10]:
alfagift_member = pd.read_json('/home/server/mongo_export/alfagift_member_01jan22.json',
                              dtype='object')
alfagift_member['sapa'] = alfagift_member['sapa'].fillna('-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: x.get('store_code') \
                                                        if x != '-' else '-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: '-' \
                                                        if x == '' else x)

alfagift_member['created_at'] = alfagift_member['created_at'].fillna('2019-01-01T23:08:53.393Z')
alfagift_member['created_at'] = alfagift_member['created_at'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)

# alfagift_member = alfagift_member[['account_card', 'sapa', 'created_at']]
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: 'sapa' if x != '-' else 'not_sapa')

alfagift_member['created_at_d'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m-%d")
alfagift_member['created_at_m'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m")

alfagift_member['account_card'] = alfagift_member['account_card'].astype('str')

In [11]:
# alfagift_member_sel = alfagift_member[(alfagift_member['created_at_m'] == '2022-01') |
#                                       (alfagift_member['created_at_m'] == '2022-02') |
#                                       (alfagift_member['created_at_m'] == '2022-03') |
#                                       (alfagift_member['created_at_m'] == '2022-04') ]
# alfagift_member_sel['created_at_d'] = pd.to_datetime(alfagift_member_sel['created_at']).dt.strftime("%Y-%m-%d")

alfagift_member_sel = alfagift_member.copy()

In [12]:
### creating label for training set


li_sus = []
for dr in pd.date_range('2022-01-01', '2022-04-17', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member_sel[alfagift_member_sel['created_at_d'] == sel_created]
    #print(alfagift_member_sel_d.shape)


    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','count'))
    alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
        alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

    sus_by_ip = pd.merge(alfagift_member_sel_d_ip, alfagift_member_sel_d, on='createdFromIp', how='left')

    ## suspect by phone
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(createdFromIp_count=('phone_sub','count'))
    alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
        alfagift_member_sel_d_phone['createdFromIp_count'] >= 5].reset_index()

    sus_by_phone = pd.merge(alfagift_member_sel_d_phone, alfagift_member_sel_d, on='phone_sub', how='left')

    
    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    
    
    
    ## saving by concating suspect aspect
    sus_all = pd.concat([
        sus_by_ip[['account_card','email','phone']], 
        sus_by_phone[['account_card','email','phone']],
        alfagift_member_sel_d[alfagift_member_sel_d['member_status'] == 27][['account_card','email']],
        alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] == 1][['account_card','email']]
    ], axis=0)\
        .drop_duplicates()
    
    sus_all = sus_all[~sus_all['email'].str.contains('deliverysapa', case=False)]
    sus_all = sus_all[['account_card','phone','email']]
    
    
    
    li_sus.append(sus_all)
#     sus_all.to_csv(
#         '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_{}'.format(sel_created)
#     )

    #break

df_sus_all = pd.concat(li_sus).drop_duplicates()
df_sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_all1.csv', index=False)

In [13]:
### checking
# alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] == 1].head(100)

In [14]:
## creating feature for training set


li_sus = []
for dr in pd.date_range('2022-01-01', '2022-04-17', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member_sel[alfagift_member_sel['created_at_d'] == sel_created]
    print("{} {}".format(sel_created, alfagift_member_sel_d.shape))


    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','count'))
#     alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
#         alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

    alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
                                     alfagift_member_sel_d_ip,
                                     on='createdFromIp', how='left').fillna(0)

    ## suspect by phone
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(phoneSub_count=('phone_sub','count'))
#     alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
#         alfagift_member_sel_d_phone['phoneSub_count'] >= 5].reset_index()

    
    
    alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
                                     alfagift_member_sel_d_phone,
                                     on='phone_sub', how='left').fillna(0)
    
    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    

    li_sus.append(alfagift_member_sel_d)
#     alfagift_member_sel_d.to_csv(
#     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily/{}'.format(sel_created))

    #break

df_sus_all = pd.concat(li_sus)
df_sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full.csv'
    , index=False)

2022-01-01 (8467, 14)
2022-01-02 (9583, 14)
2022-01-03 (11755, 14)
2022-01-04 (16174, 14)
2022-01-05 (14622, 14)
2022-01-06 (22992, 14)
2022-01-07 (14545, 14)
2022-01-08 (15639, 14)
2022-01-09 (15605, 14)
2022-01-10 (15749, 14)
2022-01-11 (20511, 14)
2022-01-12 (20162, 14)
2022-01-13 (20737, 14)
2022-01-14 (23634, 14)
2022-01-15 (24630, 14)
2022-01-16 (24138, 14)
2022-01-17 (24450, 14)
2022-01-18 (19670, 14)
2022-01-19 (47065, 14)
2022-01-20 (43596, 14)
2022-01-21 (37803, 14)
2022-01-22 (35822, 14)
2022-01-23 (36113, 14)
2022-01-24 (36523, 14)
2022-01-25 (38993, 14)
2022-01-26 (40381, 14)
2022-01-27 (42875, 14)
2022-01-28 (49247, 14)
2022-01-29 (28553, 14)
2022-01-30 (22372, 14)
2022-01-31 (19442, 14)
2022-02-01 (21509, 14)
2022-02-02 (26334, 14)
2022-02-03 (28899, 14)
2022-02-04 (30826, 14)
2022-02-05 (29101, 14)
2022-02-06 (28859, 14)
2022-02-07 (28859, 14)
2022-02-08 (29762, 14)
2022-02-09 (29773, 14)
2022-02-10 (29125, 14)
2022-02-11 (89472, 14)
2022-02-12 (28763, 14)
2022-02-13 (3

In [ ]:
## load feature and label 
df_sus_all = pd.read_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full.csv'
    , dtype='object')
df_label = pd.read_csv('./suspect_fraud/sus_all1.csv', dtype='object')
df_label['label'] = 1

## merging feature with suspect label
df_train = pd.merge(df_sus_all, df_label, on='account_card', how='left')
df_train['label'] = df_train['label'].fillna(0)

In [ ]:
# df_train[df_train['label'] == 1].head(100)

In [ ]:
wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',

]
X = df_train[wrap_col]
y = df_train[['label']]

In [ ]:
regressor = DecisionTreeClassifier(random_state=0, max_depth=4)
regressor.fit(X,y)


dump(regressor, './model/regis_dtree')
r = export_text(regressor, feature_names=list(X), decimals=0, spacing=6)
print("prediction rule \n\n{}".format(r))

In [ ]:
clf = LogisticRegression(random_state=0)
clf.fit(X, y)
dump(clf, './model/regis_logreg')

In [ ]:
## for concating feature and bobot
pd.concat([pd.DataFrame(list(X), columns=['variabel']), 
           pd.DataFrame(pd.Series(clf.coef_[0]), columns=['bobot'])], 1)\
.append({'variabel':'intercept','bobot':clf.intercept_[0]},ignore_index=True)

In [ ]:
model_clf = load('./model/regis_logreg')

In [ ]:


X_test = pd.DataFrame()
X_test['score_gibberish'] = [1]
X_test['score_sus_email'] = [0]
X_test['createdFromIp_count'] = [0]
X_test['phoneSub_count'] = [0]

score = model_clf.predict_proba(X_test)
score[0][-1]

In [ ]:
res_pred = pd.concat([
    pd.DataFrame(clf.feature_names_in_, columns=['variabel']), 
    pd.DataFrame(clf.coef_[0], columns=['bobot']),
    pd.DataFrame(X_test.values[0], columns=['feature'])      
], 1)
res_pred['b*f'] = res_pred['bobot'] * res_pred['feature']

dict_score_parameter = {}
for idx, row in res_pred.iterrows():
    dict_score_parameter[row['variabel']] = row['b*f']
dict_score_parameter

In [ ]:
{
    is_fraud:1,
    score_final:x
    score_parameter:[
        
    ],
    reason:[
        perparameter
    ]
}